In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/ner_dataset-1.csv',encoding='latin1')

In [ ]:
data.head(20)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [ ]:
data=data.fillna(method='ffill')

/tmp/ipython-input-5-4209732964.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data=data.fillna(method='ffill')


In [ ]:
words=list(set(data['Word'].values))
pos=list(set(data['POS'].values))
tags=list(set(data['Tag'].values))
print(words)
print(pos)
print(tags)

['handler', 'Caucasus', 'pegs', 'premiere', 'Djinnit', 'Babil', 'lesser-known', 'Piazza', 'Ukraine', 'France-Africa', 'Juarez', 'ghettos', 'bittersweet', 'platforms', 'incursion', 'bulldozer', '132-seat', 'Brent', 'prosper', 'Coleand', '60,000-seat', 'servicemen', 'refraining', 'renegade', 'Bajramovic', 'Segundo', 'wear', 'deliberating', 'Councilor', 'high-definition', 'anti-dumping', 'threats', '300-member', 'off-again', 'versus', 'renew', 'Fuge', 'co-star', 'glistening', 'Shanghai-bound', 'haven', 'E.U', 'dare', 'advisors', 'Zabol', 'watchful', 'etched', 'fourth-seed', 'asylum-seeker', 're-establishing', 'sanitary', 'younger', 'postponement', 'risque', 'rebuke', 'reschedule', 'Bab', 'Dublin', 'Shabelle', 'anti-corruption', 'sworn', 'Chase', 'eight', 'lowlands', 'TRUE', 'Mughani', 'cloud', 'scattering', 'Yong-Chun', 'CIS', 'Future', 'Mwesige', 'heyday', 'Bezsmertniy', 'precision', 'Rogge', 'stables', 'showcase', 'mounting', 'backtrack', '115.2', 'Cristobal', 'rob', 'rose', 'Galo', 'fi

In [ ]:
words.append('dummy')
tags.append('.')

In [ ]:
def extract_sentences(data):
    sentences = []
    for _, group in data.groupby("Sentence #"):
        # Create a list of (word, pos, tag) tuples for each sentence
        sentence = list(zip(group["Word"], group["POS"], group["Tag"]))
        sentences.append(sentence)
    return sentences

sentences = extract_sentences(data)


In [ ]:
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [ ]:
word2idx = {word: idx for idx, word in enumerate(words)}
tag2idx = {tag: idx for idx, tag in enumerate(tags)}

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len=50
x=[[word2idx[w[0]] for w in s] for s in sentences]
x=pad_sequences(maxlen=max_len,sequences=x,padding='post',value=len(words)-1)
x[0]

array([26067, 19901, 20521, 33921, 28380, 10272, 26591, 13795,  8468,
       34649,  1433, 15233, 29438, 20191,  4263, 34649,   443, 19901,
       18333, 22353, 24154, 14517, 20234,  4247, 35167, 35167, 35167,
       35167, 35167, 35167, 35167, 35167, 35167, 35167, 35167, 35167,
       35167, 35167, 35167, 35167, 35167, 35167, 35167, 35167, 35167,
       35167, 35167, 35167, 35167, 35167], dtype=int32)

In [ ]:
y=[[tag2idx[w[2]] for w in s]for s in sentences]
y=pad_sequences(maxlen=max_len,sequences=y,padding='post',value=tag2idx['.'])
y[0]

array([ 5,  5,  5,  5,  5,  5, 15,  5,  5,  5,  5,  5, 15,  5,  5,  5,  5,
        5,  7,  5,  5,  5,  5,  5, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
      dtype=int32)

In [ ]:
from tensorflow.keras import utils
y=utils.to_categorical(y,num_classes=len(tags))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Embedding,Dense,TimeDistributed,Dropout,Bidirectional,Input

In [ ]:
input = Input(shape=(max_len,)) # Input layer
model = Embedding(input_dim=len(words), output_dim=50, input_length=max_len)(input) # Word embedding layer
model = Dropout(0.1)(model) # Dropout
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model) # Bi-directional LSTM layer
out = TimeDistributed(Dense(len(tags), activation="softmax"))(model)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model = Model(input, out)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]) # Compile with an optimizer

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(x, y, test_size=0.20)

In [ ]:
import numpy as np

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=3, validation_split=0.1, verbose=1) # Train

Epoch 1/3
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 232s 206ms/step - accuracy: 0.9224 - loss: 0.3476 - val_accuracy: 0.9613 - val_loss: 0.1355
Epoch 2/3
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 264s 209ms/step - accuracy: 0.9696 - loss: 0.1090 - val_accuracy: 0.9782 - val_loss: 0.0760
Epoch 3/3
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 262s 209ms/step - accuracy: 0.9809 - loss: 0.0673 - val_accuracy: 0.9814 - val_loss: 0.0628


In [ ]:
i = 1213 # Some test sentence sample
p = model.predict(np.array([X_te[i]])) # Predict on it
p = np.argmax(p, axis=-1) # Map softmax back to a POS index
for w, pred in zip(X_te[i], p[0]):  # for every word in the sentence
    if w != len(words)-1:
      print("{:20} -- {}".format(words[w], tags[pred])) # Print word and tag

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
The                  -- O
latest               -- O
violence             -- O
comes                -- O
shortly              -- O
after                -- O
the                  -- O
chief                -- O
of                   -- O
staff                -- O
of                   -- O
Iraq                 -- B-geo
s                    -- O
armed                -- O
forces               -- O
(                    -- O
Babaker              -- O
Zibari               -- O
)                    -- O
told                 -- O
VOA                  -- B-org
(                    -- O
Persian              -- B-geo
News                 -- I-org
Network              -- I-org
)                    -- O
that                 -- O
unlike               -- O
in                   -- O
the                  -- O
past                 -- O
,                    -- O
al-Qaida             -- B-org
members              -- O
and                  -- O
other                -- O
te

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_te)
y_pred = np.argmax(y_pred, axis=-1)
y_test = np.argmax(y_te, axis=-1)
accuracy = accuracy_score(y_test.flatten(), y_pred.flatten())
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
idx2tag = {idx: tag for idx, tag in enumerate(tags)}
predicted_tags = [idx2tag[p] for p in y_pred[0]]
print(predicted_tags)